In [1]:
import pandas as pd
from ipywidgets import interact, IntRangeSlider, interactive, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

# Ensure widget display works properly in Binder
widgets.Widget.close_all()

In [2]:
def clean_df(df, extra_col):
    # Drop first row (empty)
    df = df.drop(df.index[0])

    # Add year column name
    df.iloc[0, 0] = "year"

    # Makey years the header
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])

    # Drop all rows starting with Nan
    df = df.dropna(subset=[df.columns[0]])

    # # Remove indicator group titles
    patterns_to_remove = [
        "Utemiljö",
        "Missbruksproblem", 
        "Utomhusstörningar",
        "Andel uppfattat minst ett problem",
        "Utsatthet för brott",
        "Oro för att utsättas för brott",
        "Konkret känsla av otrygghet",
        "Polisens agerande mot problem",
        "Tillit"
    ]
    mask = ~df.iloc[:, 0].astype(str).str.startswith(tuple(patterns_to_remove))
    df = df[mask]

    # Transpose df
    df = df.transpose()

    # Make the first row the header
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])

    # Reset index
    df = df.reset_index()

    # Rename first column to "year"
    df.columns.values[0] = "year"

    # Remove rows with 'Year' == '2020_1' or '2016_1'
    df = df[~df['year'].isin(['2020_1', '2016_1'])]

    # Rename '2020_2' to '2020' and '2016_2' to '2016', etc
    df['year'] = df['year'].replace({'2020_2': '2020', '2016_2': '2016', '2006*': '2006'})
    
    # Rename special characters ä, å and ö
    df.columns = df.columns.str.replace('ä', 'a').str.replace('å', 'a').str.replace('ö', 'o')

    # Rename columns to snake_casing for easier coding
    df.columns = df.columns.str.replace(' ', '_').str.replace('.', '').str.replace(',', '').str.lower()

    # Transform years to numbers
    df["year"] = df["year"].astype(int)

    # Transform numeric columns to numbers
    df = df.apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',', '.'), errors="coerce") if x.dtype == 'object' else x)

    # Add extra column
    if isinstance(extra_col, dict):
        for col_name, col_value in extra_col.items():
            df.insert(0, col_name, col_value)

    return df

# The data

In [3]:
blekinge_nordostra_skane_raw = pd.read_excel('Resultatbild - PO Blekinge nordostra Skane.xlsx')
kalmar_kronoberg_raw = pd.read_excel('Resultatbild - PO Kalmar Kronoberg.xlsx')
malmo_raw = pd.read_excel('Resultatbild - PO Malmo.xlsx')
norvastra_skane_raw = pd.read_excel('Resultatbild - PO Nordvastra Skane.xlsx')
sodra_skane_raw = pd.read_excel('Resultatbild - PO Sodra Skane.xlsx')

# Clean all raw data
blekinge_nordostra_skane_df = clean_df(blekinge_nordostra_skane_raw, {'po':'Blekinge nordostra Skane'})
kalmar_kronoberg_df = clean_df(kalmar_kronoberg_raw, {'po':'Kalmar Kronoberg'})
malmo_df = clean_df(malmo_raw, {'po':'Malmo'})
norvastra_skane_df = clean_df(norvastra_skane_raw, {'po':'Nordvastra Skane'})
sodra_skane_df = clean_df(sodra_skane_raw, {'po':'Sodra Skane'})

# Join clean dfs
df = pd.concat([
    blekinge_nordostra_skane_df,
    kalmar_kronoberg_df,
    malmo_df,
    norvastra_skane_df,
    sodra_skane_df
])

df.sample(10)

df

year,po,year,nedskrapning,skadegorelse,berusade_personer_utomhus,narkotikapaverkade_personer_utomhus,bostader_tillhall_for_alkoholmissbrukare,bostader_tillhall_for_narkotikamissbrukare,observerat_narkotikaforsaljning_i_omradet,folk_brakar_och_slass_utomhus,...,restaurang_bar_eller_disco,sportevenemang,foreningsmoten_kurser_och_liknande,åka_buss_eller_tag,andel_avstatt_fran_nagon_typ_av_aktivitet,polisen_bryr_sig_om_de_lokala_problemen,polisen_bryr_sig_inte_om_de_lokala_problemen,relationskvot,boende_skulle_ej_agera_vid_slagsmal,svag_sammanhallning_i_bostadsomradet
0,Blekinge nordostra Skane,2024,34.85,30.29,13.53,13.94,7.89,9.09,9.32,8.99,...,7.69,2.59,2.23,9.51,13.25,45.13,11.04,24.47,17.35,10.33
1,Blekinge nordostra Skane,2023,35.47,29.69,13.09,13.99,8.47,9.73,10.15,10.35,...,7.75,3.54,2.87,11.73,15.46,45.34,11.39,25.11,16.16,10.32
2,Blekinge nordostra Skane,2022,39.03,34.14,16.99,16.47,8.69,9.91,10.50,12.62,...,8.53,3.96,3.22,11.94,16.39,47.68,13.50,28.31,17.48,10.96
3,Blekinge nordostra Skane,2021,39.98,34.22,16.10,15.22,7.66,9.25,9.53,12.00,...,9.31,3.41,3.68,12.74,16.64,45.66,13.54,29.65,18.37,11.14
4,Blekinge nordostra Skane,2020,39.93,33.34,17.19,16.26,8.47,10.52,11.01,13.10,...,8.96,3.31,3.77,12.55,16.73,46.20,13.40,29.01,19.43,12.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,Sodra Skane,2011,31.76,32.09,13.28,7.94,5.47,4.82,NaN,8.15,...,4.62,2.39,1.61,6.31,10.07,43.04,11.82,27.46,NaN,NaN
13,Sodra Skane,2010,29.06,32.31,12.68,7.68,5.49,5.05,NaN,8.39,...,4.97,1.70,1.76,7.72,10.97,44.19,13.04,29.52,NaN,NaN
14,Sodra Skane,2009,31.08,33.58,14.36,8.10,5.52,4.86,NaN,8.31,...,4.02,1.91,1.83,6.41,9.69,40.44,14.15,35.00,NaN,NaN
15,Sodra Skane,2008,34.38,38.52,14.12,7.98,5.78,5.34,NaN,9.29,...,5.49,1.85,2.07,7.48,11.65,38.33,13.74,35.85,NaN,NaN


## Using all available years

In [4]:
def simple_plot(column, year_range):
    plt.style.use('bmh')
    plt.figure(figsize=(15, 6))
    
    for region in np.sort(df['po'].unique()):
        # Filter data by region and year range
        data = df[
            (df['po'] == region) & 
            (df['year'] >= year_range[0]) & 
            (df['year'] <= year_range[1])
        ]
        
        if not data.empty:
            plt.plot(data['year'], data[column], label=region)
    
    plt.title(f'{column.replace("_", " ").title()} by Region ({year_range[0]}-{year_range[1]})')
    plt.xlabel('Year')
    plt.ylabel(column.replace("_", " ").title())
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Set integer ticks for x-axis
    if len(plt.gca().get_lines()) > 0:  # Only if there are lines to plot
        years_in_range = sorted(df[
            (df['year'] >= year_range[0]) & 
            (df['year'] <= year_range[1])
        ]['year'].unique())
        plt.xticks(years_in_range)
    
    plt.tight_layout()
    plt.show()

# Get numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
columns_to_exclude = ['year']
plot_columns = [col for col in numeric_cols if col not in columns_to_exclude]
options = {f"{col.replace("_", " ").title()}": col for col in plot_columns}
dropdown = widgets.Dropdown(
    options=options,
    description="Indicator:"
)

# Get year range from data
min_year = int(df['year'].min())
max_year = int(df['year'].max())

# Create interactive widget with year range slider
year_slider = IntRangeSlider(
    value=(min_year, max_year),
    min=min_year,
    max=max_year,
    step=1,
    description='Years:',
    style={'description_width': 'initial'}
)

# Create the interactive widget and display it explicitly
interactive_plot = interactive(simple_plot, 
                              column=dropdown,
                              year_range=year_slider)

# Display the widget explicitly - this helps with Binder compatibility
display(interactive_plot)

interactive(children=(Dropdown(description='Indicator:', options={'Nedskrapning': 'nedskrapning', 'Skadegorels…

## Using 5-year-span groups

In [5]:
# Create 5-year span bins
bins = range(df['year'].min(), df['year'].max() + 5, 5)
labels = [f"{b}-{b+4}" for b in bins[:-1]]
df['year_span'] = pd.cut(df['year'], bins=bins, labels=labels, right=True)

# Group by region and 5-year span, then aggregate
main_5year_df = df.groupby(['po', 'year_span'], as_index=False).mean(numeric_only=True).round(2)

main_5year_df = main_5year_df.drop('year', axis=1)

main_5year_df.sample(10)

/var/folders/w1/f091_bwn6xj9x65pnxwjxkz40000gn/T/ipykernel_83062/220097939.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  main_5year_df = df.groupby(['po', 'year_span'], as_index=False).mean(numeric_only=True).round(2)


year,po,year_span,nedskrapning,skadegorelse,berusade_personer_utomhus,narkotikapaverkade_personer_utomhus,bostader_tillhall_for_alkoholmissbrukare,bostader_tillhall_for_narkotikamissbrukare,observerat_narkotikaforsaljning_i_omradet,folk_brakar_och_slass_utomhus,...,restaurang_bar_eller_disco,sportevenemang,foreningsmoten_kurser_och_liknande,åka_buss_eller_tag,andel_avstatt_fran_nagon_typ_av_aktivitet,polisen_bryr_sig_om_de_lokala_problemen,polisen_bryr_sig_inte_om_de_lokala_problemen,relationskvot,boende_skulle_ej_agera_vid_slagsmal,svag_sammanhallning_i_bostadsomradet
13,Nordvastra Skane,2010-2014,33.58,32.44,14.73,10.24,6.44,6.35,NaN,9.15,...,5.40,4.40,2.60,8.69,13.80,42.52,13.05,30.86,NaN,NaN
15,Nordvastra Skane,2020-2024,37.64,33.44,14.50,14.48,6.83,8.00,11.72,10.55,...,7.78,5.08,2.85,12.04,16.71,48.72,12.00,24.65,18.62,11.51
17,Sodra Skane,2010-2014,29.33,28.04,11.82,7.61,5.09,4.80,NaN,7.04,...,4.15,2.47,1.88,6.76,10.12,42.73,11.78,27.60,NaN,NaN
8,Malmo,2005-2009,45.36,45.18,23.36,18.68,10.36,10.17,NaN,19.82,...,8.70,3.65,4.06,11.08,17.04,45.08,13.25,29.66,NaN,NaN
5,Kalmar Kronoberg,2010-2014,24.90,22.45,12.61,7.28,6.15,5.35,NaN,6.96,...,3.49,1.70,1.60,4.32,7.45,41.90,12.30,29.36,NaN,NaN
2,Blekinge nordostra Skane,2015-2019,36.60,30.52,15.26,13.34,7.71,8.69,11.01,11.44,...,7.92,3.01,3.32,10.88,15.15,42.71,14.61,34.52,17.81,11.34
16,Sodra Skane,2005-2009,31.34,35.32,13.86,8.03,5.76,5.23,NaN,8.97,...,4.79,1.78,1.82,7.04,10.53,40.39,13.52,33.61,NaN,NaN
11,Malmo,2020-2024,46.72,36.93,18.82,19.46,7.85,9.77,20.23,15.08,...,8.24,5.17,3.20,12.57,17.23,54.86,11.18,20.43,21.13,16.48
9,Malmo,2010-2014,45.08,42.90,22.24,20.12,9.95,10.59,NaN,19.75,...,7.90,4.86,3.82,11.04,16.98,48.32,13.60,28.18,NaN,NaN
1,Blekinge nordostra Skane,2010-2014,28.50,24.62,13.25,8.50,6.54,5.88,NaN,7.34,...,3.53,1.30,1.53,4.46,7.39,40.95,11.92,29.14,NaN,NaN


In [6]:
def five_year_plot(column):
    plt.style.use('bmh')
    plt.figure(figsize=(15, 6))
    for region in np.sort(main_5year_df['po'].unique()):
        data = main_5year_df[main_5year_df['po'] == region]
        plt.plot(data['year_span'], data[column], label=region)
    plt.title(f'{column.replace("_", " ").title()} by Region (5-Year Spans)')
    plt.legend()
    plt.show()

numeric_cols = main_5year_df.select_dtypes(include=[np.number]).columns.tolist();
options = {f"{col.replace("_", " ").title()}": col for col in numeric_cols}
dropdown = widgets.Dropdown(
    options=options,
    description="Indicator:"
    
)
# Create and display the interactive widget explicitly
interactive_5year_plot = interactive(five_year_plot, column=dropdown)
display(interactive_5year_plot)

interactive(children=(Dropdown(description='Indicator:', options={'Nedskrapning': 'nedskrapning', 'Skadegorels…

# TODO
* Explore yearly changes (delta) rather than index number (which region has changed the most in certain periods?)
* Opportunities for scatter plot

# DONE
* Create interactive charts to visualize how the 5 regions compare over time
* Create a way to visualize buckets of 5 years at a time instead of all years
* Find interesting correlations between indicators
* Success signs
* Warning signs
* Surprising signs

In [ ]:
# Save widget state for Binder compatibility
# This helps ensure widgets display properly when the notebook is shared
try:
    from ipywidgets import embed
    # This will embed the widget state in the notebook
    embed.embed_minimal_html('widgets.html', [interactive_plot, interactive_5year_plot], title='Survey Data Visualization')
    print("Widget state saved successfully for Binder compatibility")
except Exception as e:
    print(f"Note: Widget state saving not available: {e}")
    print("Widgets should still work in interactive environments")
